In [ ]:
%cd ..

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import plotly.graph_objects as go
import preprocessing
import metrics
import plotting_utils
from hyperparameter_tuner import LinearSVCHyperParameterTuner, KNNHyperParameterTuner, LGBMHyperParameterTuner
import pickle
from PIL import Image

# DATA

Load data

In [ ]:
original_df = pd.read_csv("data/Churn_Modelling.csv", index_col=0)

Prepare data (remove uninformative columns and convert string to one hot encoding)

In [ ]:
df = preprocessing.drop_columns_and_convert_strings(original_df)

Look at the correlations

In [ ]:
plotting_utils.plot_correlations(df)

Look at pairplot (quite slow, so loading precomputed one by default)

In [ ]:
precomputed = True
if not precomputed:
    sns_plot = sns.pairplot(df, hue="Exited")
    sns_plot.figure.savefig("pairplot.jpg")
Image.open("pairplot.jpg")

From these two plots, it seems Age (high correlation) and NumOfProducts (high NumOfProducts are almost always Exited in the pairplot) are important features. We will see if this is confirmed later in the models.

# Training

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, stratify=df["Exited"], random_state=2)
X_train, y_train = preprocessing.split_label(df_train)
X_test, y_test = preprocessing.split_label(df_test)


To guide our exploration, we will use two metrics:
1) Precision at given recall. This one is more business oriented. Since we are predicting churn, we want to have a high recall (we don't want to miss customers that are likely to exit). We could choose with the concerned product manager an acceptable rate of recall (I will assume 90% in the following). The metric we will optimize is therefore the precision corresponding to that recall. Higher precision means fewer False Positive. This approach assumes the procedure when a customer is predicted as churn is not too high (there will be a high number of false positive). It seems reasonable for the case where we would just send an email to that customer.

2) The area under the curve of the Receiver Operating Characteristic (roc_auc). This is a more generic metric, but not uncorrelated with the first one (higher roc_auc will mean higher precision). It's one that's more familiar to ML engineers and therefore could speak more to their intuition.

Precision at 90% recall is the metric we will optimize, while still keeping an eye on roc_auc.

### Random classifier

To get an idea of whether ML actually makes an improvement, we'll see what type of scores we get from a random classifier (it assigns a random score between 0 and 1)

In [ ]:
from random_classifier import RandomClassifier
       
random_model = RandomClassifier()
metrics.get_precision_and_roc_auc(X_test, y_test, random_model)

### Linear SVC model (baseline)

As a baseline, we will use a simple linear svc model. We'll gradually complexify the models (and/or preprocessing) hoping to improve on this baseline.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

linear_svc = Pipeline([('scaler', MinMaxScaler()), ('clf', LinearSVC(dual=False))])
linear_svc.fit(X_train, y_train)

metrics.get_precision_and_roc_auc(X_test, y_test, linear_svc)

It seems we are doing much better than the random classifier (35% increase in precision)

In [ ]:
coefs = linear_svc.named_steps.clf.coef_[0]
plotting_utils.plot_importance(coefs, X_train.columns)

We find that Age is indeed an important feature as expected from the correlation analysis. IsActiveMember is also quite important, with a negative correlation. This makes sense, as active member should be less inclined to churn

### Tuned Linear SVC model

In [ ]:
linear_svc_tuner = LinearSVCHyperParameterTuner()
tuned_linear_svc = linear_svc_tuner.get_tuned_model(df_train, 20)
metrics.get_precision_and_roc_auc(X_test, y_test, tuned_linear_svc)

The tuned model doesn't do significantly better. Let's move on to more complex algorithms

### KNN

KNN seems like a good model for churn: if a customer A churns, we can guess that a customer B similar to A is likely to churn as well

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = Pipeline([('scaler', MinMaxScaler()), ('clf', KNeighborsClassifier())])

knn.fit(X_train, y_train)

metrics.get_precision_and_roc_auc(X_test, y_test, knn)

The choice of n_neighbors in KNeighborsClassifier is highly data dependent. To find the optimal one we will run a 1D simple hyperparameter optimization using Optuna (not really necessary here, but since we will use it for later models, we might as well use it here also)

In [ ]:
knn_tuner = KNNHyperParameterTuner()
tuned_knn = knn_tuner.get_tuned_model(df_train, 2)
metrics.get_precision_and_roc_auc(X_test, y_test, tuned_knn)

It's not necessarily better. However, drawing conclusion from a single test set is dangerous. We'll do a more careful model comparison later

### Base LGBM

Gradient boosted trees are strong models for tabular data. In particular, the lightgbm library provides a fast implementation

In [ ]:
from lightgbm import LGBMClassifier

base_lgbm = LGBMClassifier()
base_lgbm.fit(X_train, y_train)
metrics.get_precision_and_roc_auc(X_test, y_test, base_lgbm)

This is quite an increase compared to our baseline {'precision_at_90.0_recall': 0.2718532794068828, 'roc_auc': 0.7594163914432234}

In [ ]:
plotting_utils.plot_importance(base_lgbm.feature_importances_, base_lgbm.feature_name_)

Age is again our top feature. Here we see that NumOfProducts is also high in the list, as expected from the pairplot analysis.

### Recursive Feature Elimination

Models such as boosted trees with provide feature importances can be use to remove less informative features.
Implementation is based on sklearn.feature_selection.RFECV 

In [ ]:
from recursive_feature_elimination import ModelRecursiveFeatureElimination
rfeliminator = ModelRecursiveFeatureElimination()
tree_classifier_rfe = rfeliminator.get_model(X_train, y_train, LGBMClassifier())
tree_classifier_rfe.fit(X_train, y_train)
metrics.get_precision_and_roc_auc(X_test, y_test, tree_classifier_rfe)

In [ ]:
plotting_utils.plot_importance(tree_classifier_rfe.named_steps.clf.feature_importances_, rfeliminator.cols_to_keep)

There doesn't seem to be any significant improvement from the RFE

### Tuned LGBM

LGBM has a lot of hyperparameters, choosing the right ones can make a big difference. We're going to do some exploration with Optuna, optimizing for precision at 90% recall.

In [ ]:
# Doing the tuning takes some time (typically 5min), can load trained model instead
do_tuning = False
if do_tuning:
    lgbm_tuner = LGBMHyperParameterTuner()
    tuned_lgbm = lgbm_tuner.get_tuned_model(df_train, n_trials=100)
else:
    tuned_lgbm = pickle.load(open("inference/model/tuned_lgbm.pickle", "rb"))
print("\n On test set, we have")
metrics.get_precision_and_roc_auc(X_test, y_test, tuned_lgbm)

In [ ]:
plotting_utils.plot_importance(tuned_lgbm.feature_importances_, tuned_lgbm.feature_name_)

This seems to be the best candidate so far. We'll see if a more robust analysis confirms it.

## Meta Model

Another option is to combined the previous models into a Meta Model. Since we want to compute precision for a given recall, we need to have a score (ie a float), not just a prediction (in (0, 1)). In order to combine scores which comes for very different models, we will map the scores to the corresponding precision value, using an interpolated function computed on the training data to avoid data leakage. More details in the meta_model python file.

In [ ]:
dict_models = {
    "linear_svc": linear_svc,
    "tuned_linear_svc": tuned_linear_svc,
    "knn": knn,
    "tuned_knn": tuned_knn,
    "base_lgbm": base_lgbm,
    "tree_classifier_rfe": tree_classifier_rfe,
    "tuned_lgbm": tuned_lgbm,
}

In [ ]:
from meta_model import MetaModel

meta_model = MetaModel(dict_models)
meta_model.fit(X_train, y_train)
metrics.get_precision_and_roc_auc(X_test, y_test, meta_model)

This doesn't seem to be much better.

## Feature Engineering

Feature engineering can be a powerful tool to help machine learning models. However, nothing really jumps out from the pair plot I first did. In the absence of more information about the business to guide our intuition, we would need to resort to blind exploration. This can be very time consuming so I decided to focus on other aspects.

## Model Comparison

To determine whether model A is significantly better than model B, it's not sufficient to compare the performance on a given test set. It could very well be that the uncertainty on the model performance (coming from the fact that the test set has a finite size) is larger than the difference in performance. In order to robustly assess which model is better, we need train and evaluate on many different train test splits. If a model is better than the others most of the time, then we can be more confident it will be better once in production

In [ ]:
from model_comparison import ModelComparator
from copy import deepcopy
new_dict_models = {
   "random_model": random_model,
   "linear_svc": linear_svc,
   "tuned_linear_svc": tuned_linear_svc,
   "knn": knn,
   "tuned_knn": tuned_knn,
    "base_lgbm": base_lgbm,
   "tree_classifier_rfe": tree_classifier_rfe,
    "tuned_lgbm": tuned_lgbm,
   "meta_model": meta_model
}


In [ ]:
plotting_utils.plot_precision_recall_multiples(X_test, y_test, new_dict_models)

In [ ]:
model_comparator = ModelComparator(new_dict_models, n_tries=100)
precision_scores, auc_scores = model_comparator.compare_models(*preprocessing.split_label(df))

In [ ]:
fig = go.Figure(data=[go.Histogram(x=precision_scores[col], nbinsx=30, name=col) for col in precision_scores if col != "best"])
fig.show()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=auc_scores[col], nbinsx=30, name=col) for col in precision_scores if col != "best"])
fig.show()

In [ ]:
best_model = precision_scores["best"].value_counts().index[0]
baseline_model_name = "linear_svc"
comparison_summary = model_comparator.compare_score_to_baseline(precision_scores, baseline_model_name)

percentage_time_better = round(100*(comparison_summary.loc["ratio_of_wins", best_model]))
mean_improvement = round(100*(comparison_summary.loc["change_in_score", best_model]))
print(f"Model {best_model} is better than {baseline_model_name} {percentage_time_better}% of the time, changing precision by on {mean_improvement}% average ")

It seems the best performing model is the tuned_lgbm model, which we will explore further 

In [ ]:
# import pickle
# pickle.dump(tuned_lgbm, open("inference/model/tuned_lgbm.pickle", "wb"))
scores = metrics.get_scores(X_test, tuned_lgbm)
threshold, _ = metrics.get_threshold_and_precision_at_recall(y_test, scores)
print(f"Threshold for model is {round(threshold,2)}")

## Error Analysis

We will associate a status to each prediction showing whether it is a True Positive (TP), True Negative (TN), False Positive (FP), False Negative (FN)

In [ ]:
df_test_status = metrics.get_confusion_status(X_test, y_test, tuned_lgbm)
plotting_utils.plot_confusion_matrix(X_test, y_test, tuned_lgbm)

We want to explore visually if we can separate false positive from true positive. For that, we reduce the dimensionality to 2 using PCA and sample 450 points of each TP and FP

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
df_test_positive = df_test_status.loc[df_test_status['status'].isin(["TP", "FP"])]
df_test_sample = df_test_positive.groupby("status").sample(450, random_state=1)
dimred_pipeline = Pipeline([("scaler", StandardScaler()), ("dimred", PCA(2))])
X_test_scaled = dimred_pipeline.fit_transform(df_test_sample[X_test.columns])
status = df_test_sample["status"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
x, y = X_test_scaled.T

fig = go.Figure(data=[go.Scatter(
    x=x,
    y=y,
    mode='markers',
    hovertext=status,
    marker=dict(
        size=12,
        color=LabelEncoder().fit_transform(status), 
        colorscale='Viridis', 
        opacity=0.4
    )
)])

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

There doesn't seem to be any region where a status dominates. 

In [ ]:
if not precomputed:
    sns_error_plot = sns.pairplot(df_test_sample[tuned_lgbm.feature_name_+['status']], hue="status", plot_kws={'alpha': 0.3})
    sns_error_plot.figure.savefig("error_pairplot.jpg")
Image.open("error_pairplot.jpg")

There are no plots where we can clearly distinguish between TP and FP

### Explore extra features

Since Balance and NumOfProducts seem to be important features, a natural feature we could add is the average balance per product.

In [ ]:
transformer = preprocessing.DivideColumns("Balance", "NumOfProducts")
df_extra = transformer.fit_transform(df_train)
do_tuning = False
if do_tuning:
    lgbm_tuner = LGBMHyperParameterTuner()
    tuned_lgbm_extra = lgbm_tuner.get_tuned_model(df_extra, n_trials=100)
else:
    tuned_lgbm_extra = pickle.load(open("inference/model/tuned_lgbm_extra.pickle", "rb"))

In [ ]:
extra_feature_lgbm = Pipeline([("transformer",transformer), ("clf", tuned_lgbm_extra) ])
extra_feature_lgbm.fit(X_train, y_train)
metrics.get_precision_and_roc_auc(X_test, y_test, extra_feature_lgbm)

In [ ]:
model_comparator = ModelComparator({"tuned_lgbm":tuned_lgbm,"extra_feature_lgbm":extra_feature_lgbm} , n_tries=100)
precision_scores, auc_scores = model_comparator.compare_models(*preprocessing.split_label(df))

In [ ]:
comparison_summary = model_comparator.compare_score_to_baseline(precision_scores, "tuned_lgbm")

percentage_time_better = round(100*(comparison_summary.loc["ratio_of_wins", "extra_feature_lgbm"]))
mean_improvement = round(100*(comparison_summary.loc["change_in_score", "extra_feature_lgbm"]))
print(f"Model extra_feature_lgbm is better than tuned_lgbm {percentage_time_better}% of the time, changing precision by on {mean_improvement}% average ")

It seems to be performing marginally worse. To see if the extra feature we added had an impact (and if that impact was the one we wanted) we can turn to SHAP values. Essentially, shap values tell us how much has each feature contributed to the prediction compared to the average prediction. Without going into the details, this is roughly done by replacing a given feature value by a random value and seeing how much it changes the outcome

In [ ]:
import shap
shap.initjs()
model = extra_feature_lgbm.named_steps.clf
explainer = shap.TreeExplainer(model)
expected_value = explainer.expected_value

In [ ]:
features = extra_feature_lgbm.named_steps.transformer.transform(df_test_status)[model.feature_name_]

explainer = shap.Explainer(model, features)
expected_value = explainer.expected_value
if isinstance(expected_value, list):
    expected_value = expected_value[1]
print(f"Explainer expected value: {expected_value}")

Let's look at the false positives, as they are the ones we want to reduce (since we fixed the recall at 90%)

In [ ]:
df_given_status = df_test_status.loc[lambda df: df["status"]=="FP"]
# We sort by descending proba so that the first row is the hardest false positive (the one with the lowest score)
df_given_status = df_given_status.sort_values("proba",ascending=False)
features = features.loc[df_given_status.index]
shap_values = explainer(features)
shap_interaction_values = explainer.shap_interaction_values(features)
if isinstance(shap_interaction_values, list):
    shap_interaction_values = shap_interaction_values[1]

In [ ]:
shap.plots.waterfall(shap_values[0])

We can see that indeed the extra features (Balance/NumOfProducts) pulls the prediction to lower values, but not enough to go below the threshold.